In [7]:
#import cx_Oracle
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

In [15]:
data = {}

dbtables = ['ONI_CIRCUITS', 'MEB_CONTAINERS', 'MEB_DGM', 'MEB_DMC', 'MEB_DMC_GRADE', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS', 'MEB_KS_WZORC']
username = 'dmcro'
password = 'ouZ9Rys7astDp4byjUvO'
dsn = 'vwpnwrud0000rac:1521/dmc.vwg'
dbhostname='vwpnwrud0000rac'
service_name='dmc.vwg'

In [24]:
try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table in dbtables:
        query = f"select * from z3dmc.{table} where rownum<101"
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

In [26]:
data['ONI_CIRCUIT'].head()

(99, 30)

In [11]:
"""connection=cx_Oracle.connect(username,password,dsn,encoding='UTF-8')
connection.version
connection.close()"""

In [12]:
"""try:
    with cx_Oracle.connect(username,password,dsn,encoding='UTF-8') as connection:
        with connection.cursor() as cursor:
            cursor.execute(dmc_sql)
            while True:
                row = cursor.fetchone()
                if row is None:
                    break
                print(row)
                data={'id':row[0],'dmc':row[1]}
    cursor.close()
    connection.close()
except cx_Oracle.Error as error:
    print(error)"""

(2071, '0MH301103CVW41910090437321102S111910040418', 1, datetime.datetime(2019, 10, 9, 8, 33, 13), datetime.datetime(2019, 11, 6, 6, 49, 5), 504, datetime.datetime(2019, 11, 6, 6, 49, 4), None, None, 700)
(2070, '0MH301103CVW41910090437211102S111910040342', 2, datetime.datetime(2019, 10, 9, 8, 32, 53), datetime.datetime(2019, 11, 6, 9, 14, 37), 366, datetime.datetime(2019, 11, 6, 9, 14, 36), None, None, 700)
(2069, '0MH301103CVW41910090437121102S111910040330', 107, datetime.datetime(2019, 10, 9, 8, 32, 26), datetime.datetime(2019, 11, 12, 6, 32, 27), 366, datetime.datetime(2019, 11, 12, 6, 32, 26), None, None, 700)
(2068, '0MH301103CVW41910090437011102S111910040351', 1, datetime.datetime(2019, 10, 9, 8, 18, 26), datetime.datetime(2019, 11, 6, 5, 26, 55), 504, datetime.datetime(2019, 11, 6, 5, 26, 54), None, None, 700)
(2067, '0MH301103CVW41910090436721102S111910040345', 2, datetime.datetime(2019, 10, 9, 8, 18, 5), datetime.datetime(2019, 10, 9, 8, 22, 31), 0, None, None, None, None)
(2